## Class 5 - Bagging and Boosting

### Recap of lecture and introductory remarks
In yesterday's lecture, we introduced bagging and boosting as two techniques to reduce variance and reduce variance of decision trees. Bagging and boosting are not specific to decision trees, but we will see them in action with this kind of model.

We used bagging to train a set of small decision trees (weak learners) on subsets of the training data, whose individual predictions we aggregate to make a single prediction. The resulting model is an _ensemble_ model. We have seen that `Random Forests` are popular learning algorithms that combine bagging with random sampling of features, to induce diversity in decision trees and further regularization.

On the other hand, boosting consists in training a sequence of decision trees which iteratively reduce the error of the previous decision tree because they are fitted on the residuals or on the gradients of the previous tree. We have focused specifically on `gradient boosting` and indicated `XGBoost` as a particularly powerful implementation of boosting + bagging.

Today, we will go back to the bike data, and fit `RandomForest` and `XGBoost` models, comparing their performances to those of models fitted previously. We will implement Random Forests using `scikit-learn`, and XGBoost using the XGBoost package: https://xgboost.readthedocs.io/en/stable/ 

**Note**: As last week, under `nbs/class_05` you will find a notebook called `example.ipynb`, where I provide an example of how to run today's exercise on sample data.

### Operational remarks
Two suggestions on how to go about this, based on where you are at regarding exercises from previous weeks.

1. If you have done exercises from class 2 and 3, you will have one/two notebooks with baseline, linear, KNN, and linear regularized models, as well as records of performances (which will be handy to compare performances of our new methods). In this case, my suggestion would be to work on a new notebook where you only fit the new models, and load the performance of the old models for comparison.

2. If you have not done exercises from previous classes, you have three options:
- Work on a new notebook where you only fit the models we work on today (random forest and XGBoost). Optionally, you can "manually" compare the performance of your new models to plots from previous weeks
- Work on a new notebook and also implement a couple of models from previous weeks for comparison

### Today's exercise
Work in groups on the following tasks

1. Fit a `Random Forest` model to the data (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor), using cross-validation to define the best possible range of parameters
    - There are a number of parameters that should be passed to the estimator. Carefully read the documentation, and identify a few hyperparameters you might want to manipulate
    - Define a series of possible values for these hyperparameters, and store this information into a Python dictionary. For each hyperparameter, the dictionary should include the name of the hyperparameters (as a string) as `key`, and a list including a range of possible values as `value`
    - Pass your estimator and the parameter grid to `GridSearchCV`: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html and fit this object to your training set. If you have defined *a lot* of possible values, you can consider using `RandomizedSearchCV`: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html. **Note**: you need to pass something appropriate as the value of the `scoring` argument
    - Try to answer the following questions:
        - What is `GridSearchCV` doing?
        - What is the difference between `RandomizedSearchCV` and `GridSearchCV`?
        - **Bonus question**: Given that we do have a validation set, could we do model selection without using cross-validation? Which parameter in `GridSearchCV` or `RandomizedSearchCV` would you have to change, and how, to do so?
    - Find out which hyperparameters gave the best result
        - **Hint**: look at the `.best_estimator_` attribute on a fitted `GridSearchCV`/`RandomizedSearchCV` and `.get_params()`
    - Compute the performance of this model on the training, validation, and test set
    - Compute and plot feature importances for the resulting model. You can look at the `.feature_importances_` attribute of the best estimator.
        - **Bonus question**: which method is used by default to compute feature importances? Is any other method available in `sklearn`?

2. Do the exact same things as 1., this time using `XGBRegressor` (https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBRegressor)
    - Note: you will have to install `xgboost` (https://xgboost.readthedocs.io/en/stable/install.html) to run this (in short `pip install xgboost`)
    - You will have to define an appropriate `scoring` parameter
    - Parameters for grid/randomized search will be slightly different: look at the documentation for XGBRegressor, and make informed choices based on what we discussed in class

3. Plot the performance of the best Random Forest models and the best XGBoost models, against models you fitted previously
    - Which models perform best?
    - How does the performance profile of RandomForest compare to XGBoost? Why?

4. Compare feature importances across `RandomForest` and `XGBoost`: do they look similar/different?

5. Overall reflection on modeling process
    - Reflect back on your choices for previous models: should you have transformed any of the features before fitting Linear Regression, KNN, or regularized models?
    - Can you think of ways in which our predictive problem can be made more interesting from a business perspective?
    - Which aspect of the data are we *not* modeling, that we could/should model?


### Extra tasks
- Estimate a `DecisionTreeRegressor` with cross-validation, using the same logic we applied above: how does the performance of the resulting model compare to `RandomForestRegressor` and `XGBoost` regressor?
- Go back to your fitted `GridSearchCV` or `RandomizedSearchCV`, and inspect their attributes. Can you plot performance against values of each of the parameters you are fitting? Is there any systematic pattern?
- Reflect on hyperparameters passed to `GridSearchCV` or `RandomizedSearchCV`: how do you expect that individual manipulations of these parameters would affect the bias/variance profile of your models?



In [50]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [51]:
train = pd.read_csv(f'group-RMDS/data/train_bikes.csv', index_col=0)
val = pd.read_csv(f'group-RMDS/data/val_bikes.csv', index_col=0)
test = pd.read_csv(f'group-RMDS/data/test_bikes.csv', index_col=0)

In [52]:
train.columns

Index(['dteday', 'yr', 'hr', 'holiday', 'workingday', 'temp', 'atemp', 'hum',
       'windspeed', 'casual', 'registered', 'cnt', 'proportion_casual_reg',
       'season_1', 'season_2', 'season_3', 'season_4', 'mnth_1', 'mnth_2',
       'mnth_3', 'mnth_4', 'mnth_5', 'mnth_6', 'mnth_7', 'mnth_8', 'mnth_9',
       'mnth_10', 'mnth_11', 'mnth_12', 'weekday_0', 'weekday_1', 'weekday_2',
       'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weathersit_1',
       'weathersit_2', 'weathersit_3', 'weathersit_4'],
      dtype='object')

In [53]:
# create X_splits 
redundant_cols = ["proportion_casual_reg", "registered", "casual", "cnt", "dteday", "yr"]
X_train = train.drop(redundant_cols, axis=1)
X_val = val.drop(redundant_cols, axis=1)
X_test = test.drop(redundant_cols, axis=1)

# create y_splits 
y_train = train["cnt"].values
y_val = val["cnt"].values
y_test = test["cnt"].values

In [54]:
## code from 02 - 03
def evaluate(model, X, y, nsplit, model_name, constant_value=None):
    ''' Evaluates the performance of a model 
    Args:
        model (sklearn.Estimator): fitted sklearn estimator
        X (np.array): predictors
        y (np.array): true outcome
        nsplit (str): name of the split
        model_name (str): string id of the model
        constant_value (int or None): relevant if the model predicts a constant
    '''
    performances = []

    if constant_value is not None:
        preds = np.array([constant_value] * y.shape[0])
    else:
        preds = model.predict(X)
    r2 = r2_score(y, preds)
    performance = np.sqrt(mean_squared_error(y, preds))
    performances.append({'model': model_name,
                         'split': nsplit,
                         'rmse': performance.round(4),
                         'r2': r2.round(4)})

    return performances

def evaluate_across_splits(model, X_splits, y_splits, split_names, model_name, constant_value=None):
    '''
    Evaluates the performance of a model across different splits
    Args:
        model (sklearn.Estimator): fitted sklearn estimator
        X_splits (list): list of arrays containing predictors for each split
        y_splits (list): list of arrays containing true outcome for each split
        split_names (list): list of split names
        model_name (str): string id of the model
        constant_value (int or None): relevant if the model predicts a constant
    Returns:
        list: list of performance metrics for each split
    '''
    performances = []

    for X, y, split_name in zip(X_splits, y_splits, split_names):
        performance_metrics = evaluate(model, X, y, split_name, model_name, constant_value)
        performances.extend(performance_metrics)

    return performances

## Random Forests on the Data

In [55]:
params = {
    "n_estimators": [10, 50, 100, 200], # number of trees
    "criterion": ["squared_error", "absolute_error"], # fn to measure quality of split
    "max_depth": [2, 5, 8, None], # max depth of tree 
    "min_samples_split": [2, 0.1, 0.3], # min. number of samples required to split a node. default of 2 (int = amount or float = percentage)
}

# init estimator  
estimator = RandomForestRegressor(random_state=3)

# init grid search obj.
grid = GridSearchCV(estimator, param_grid=params, cv=5, verbose=2)

In [56]:
# fit 
grid.fit(X_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] END criterion=squared_error, max_depth=2, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END criterion=squared_error, max_depth=2, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END criterion=squared_error, max_depth=2, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END criterion=squared_error, max_depth=2, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END criterion=squared_error, max_depth=2, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END criterion=squared_error, max_depth=2, min_samples_split=2, n_estimators=50; total time=   0.3s
[CV] END criterion=squared_error, max_depth=2, min_samples_split=2, n_estimators=50; total time=   0.3s
[CV] END criterion=squared_error, max_depth=2, min_samples_split=2, n_estimators=50; total time=   0.3s
[CV] END criterion=squared_error, max_depth=2, min_samples_split=2, n_estimators=50; total time=   0.3s
[C

KeyboardInterrupt: 

In [ ]:
# extract best estimator
grid.best_estimator_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': 8,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 0.1,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 50,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 3,
 'verbose': 0,
 'warm_start': False}

In [ ]:
best_estimator = grid.best_estimator_

In [ ]:
performances = evaluate_across_splits(model=best_estimator, 
                                            X_splits=[X_train, X_val, X_test],
                                            y_splits=[y_train, y_val, y_test],
                                            split_names=["train", "val", "test"],
                                            model_name="randomforest",
                                            )

In [ ]:
performances

[{'model': 'randomforest', 'split': 'train', 'rmse': 45.176, 'r2': 0.6623},
 {'model': 'randomforest', 'split': 'val', 'rmse': 59.7168, 'r2': 0.3896},
 {'model': 'randomforest', 'split': 'test', 'rmse': 55.8289, 'r2': 0.4294}]